<a href="https://colab.research.google.com/github/ManasaShankaraMurthy/Market-Basket-Analysis/blob/main/MARKET_BASKET_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 51.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=6af8833aa43611cd4713e77e5dad1e9197250f70555db59dc4fafdc9ebcbd4bf
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
!pip uninstall mlxtend

Found existing installation: mlxtend 0.14.0
Uninstalling mlxtend-0.14.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/mlxtend-0.14.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/mlxtend/*
Proceed (y/n)? y
  Successfully uninstalled mlxtend-0.14.0


In [3]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.6 MB/s 


## Importing necessary libraries

In [4]:
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import time

from mlxtend.preprocessing import TransactionEncoder


## Downloading Kaggle Dataset

In [5]:
#kaggle api token
import os
os.environ["KAGGLE_USERNAME"] = "manasasmurthy"
os.environ["KAGGLE_KEY"] = "981daa73d3babd6acc976c11d5751d3b"

In [6]:
!kaggle datasets download bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows --unzip

100% 12.6G/12.6G [01:39<00:00, 151MB/s]
100% 12.6G/12.6G [01:39<00:00, 137MB/s]


In [7]:
 # decompressing the file and saving it for future use

import gzip
import shutil
with gzip.open('/content/UkraineWar/UkraineWar/0401_UkraineCombinedTweetsDeduped.csv.gzip', 'rb') as f_in:
    with open('/content/tweets.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [8]:
#downloading stopwords

#file contains a dictionary
# key   : language code
# value : list of stopwords

!kaggle datasets download manasasmurthy/stopwords --unzip

  0% 0.00/85.6k [00:00<?, ?B/s]
100% 85.6k/85.6k [00:00<00:00, 46.2MB/s]


In [9]:
import pickle
with open('/content/STOPWORDS.pk','rb') as f:
    stopwords=pickle.load(f)

In [10]:
 #reading the saved file
data=pd.read_csv('/content/tweets.csv')
data=data[['text','language']]

In [11]:
#Creating dictionary to record time taken by the model

result={'language':[],'algorithm':[],'supp':[],'conf':[],'time':[]}

In [12]:
#spark session
spark = SparkSession.builder.master("local[1]").appName("MarketBasketAnalysis").getOrCreate()

# Main


CODES:
['en', 'ar', 'es', 'tr', 'fr', 'no', 'uk', 'hi', 'de', 'und', 'it','zh', 'in', 'pt', 'ja', 'nl', 'th', 'cs', 'bn', 'ur', 'pl', 'sv','eu', 'ru', 'sl', 'et', 'el', 'ro', 'ht', 'fa', 'da', 'or', 'lt','iw', 'gu', 'tl', 'bg', 'fi', 'lv', 'ko', 'ca', 'cy', 'ta', 'te','hu', 'mr', 'sr', 'am', 'ne', 'ml', 'vi', 'is', 'kn', 'ka', 'pa','my', 'ckb', 'hy', 'si', 'ps', 'dv']

In [235]:

language=input("enter language code: ")


enter language code: ru


In [236]:
df=data[data.language==language]
df=df.drop('language',axis=1)

#for faster computation
if df.shape[0]>5000:
  df=df.sample(n=5000)

In [237]:
spark_df=spark.createDataFrame(df)

# Text Preprocessing 

In [238]:
"""
converting entire text to lowercase
removing punctuation marks
removing emojis
removing hyperlinks
removing extra spaces 
removing stopwords

"""

spark_df=spark_df.select("*", F.lower("text").alias('text1'))
spark_df=spark_df.withColumn("text2", F.regexp_replace("text1", "[!,@,#,$,%,^,&,*,;,:,',.,<,>,?,’,‘,“,`,”,–,~,‼,⇅,\",\+,\-,=,\d,\|]", ""))
spark_df=spark_df.withColumn("text3", F.regexp_replace("text2", "\n", " "))
spark_df=spark_df.withColumn("text4", F.regexp_replace("text3", "_", " "))

spark_df=spark_df.withColumn('text5',F.regexp_replace("text4", "[((\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f))]+", ""))


spark_df=spark_df.withColumn("text6",F.trim("text5"))
spark_df=spark_df.withColumn("text7", F.regexp_replace("text6", "\s+", " "))



#stopwords are not available for few languages

if language in list(stopwords.keys()):
  remove_stop = F.udf(lambda x: ' '.join([word for word in list(set(x.split())) if not (word in stopwords[language] or word.startswith('http') or len(word)<3) ]))
else:
  remove_stop = F.udf(lambda x: ' '.join([word for word in list(set(x.split())) if not (word.startswith('http')  or len(word)<3 )]))  

spark_df=spark_df.withColumn('text8', remove_stop(spark_df.text7)) 
spark_df = spark_df.withColumn("text8", F.split("text8", " "))

spark_df=spark_df.drop('text','text1','text2','text3','text4','text5','text6','text7')
spark_df=spark_df.withColumnRenamed('text8', 'text')

spark_df.show()

<>:12: DeprecationWarning:

invalid escape sequence \+

<>:20: DeprecationWarning:

invalid escape sequence \s

<>:12: DeprecationWarning:

invalid escape sequence \+

<>:20: DeprecationWarning:

invalid escape sequence \s

<>:12: DeprecationWarning:

invalid escape sequence \+

<>:20: DeprecationWarning:

invalid escape sequence \s

<ipython-input-238-c12a0b6781e9>:12: DeprecationWarning:

invalid escape sequence \+

<ipython-input-238-c12a0b6781e9>:20: DeprecationWarning:

invalid escape sequence \s



+--------------------+
|                text|
+--------------------+
|[следующая, войну...|
|[встречают, новос...|
|[населенных, силе...|
|[stopwar, поширює...|
|[итгэлийг, нефть,...|
|[уничтоженные, во...|
|[вячеслав, спецна...|
|[итгэлийг, нефть,...|
|[україна, вертолё...|
|[недели, десятки,...|
|[войска, оказывае...|
|[тепло, прогреетс...|
|[cyberwar, cybers...|
|[обл, запорожская...|
|[пытались, сеном,...|
|[вячеслав, спецна...|
|[russianwarcrimes...|
|[көрме, stoprussi...|
|[лео, russianwarc...|
|[ненавидеть, тако...|
+--------------------+
only showing top 20 rows



In [239]:
#Transaction encoder for mlxtend models

text_list=list(spark_df.select('text').toPandas()['text'])

te=TransactionEncoder()
te_array=te.fit(text_list).transform(text_list)
txn=pd.DataFrame(te_array,columns=te.columns_)


# Frequency of the words

In [240]:

words = (spark_df.select(F.explode(spark_df.text).alias('word')))
word_count=words.groupBy('word').count()
word_count=word_count.orderBy("count", ascending=0)
word_count.show()


+-------------------+-----+
|               word|count|
+-------------------+-----+
|            ukraine| 1667|
|russianukrainianwar| 1081|
|         ukrainewar|  997|
|            украина|  966|
|         stoprussia|  870|
|  ukrainerussianwar|  849|
|     войнасукраиной|  537|
|    вторжениероссии|  533|
|     агрессияроссии|  511|
|        войнапутина|  484|
|                всу|  480|
|            украине|  420|
|           белгород|  408|
|             russia|  388|
|   russiaukrainewar|  332|
|         оккупантов|  323|
|   standwithukraine|  316|
|         российские|  267|
|   ukrainerussiawar|  264|
|               news|  260|
+-------------------+-----+
only showing top 20 rows



# **Using mlxtend library : **

# 1. Apriori Implementation






In [247]:

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

supp=float(input("Enter min support    : "))
conf=float(input("Enter min confidence : "))

start = time.time()
freq_ap=apriori(txn, min_support=supp,max_len=3, use_colnames=True)
end = time.time()
rules_apriori=association_rules(freq_ap,metric="confidence",min_threshold=conf)
result['language'].append(language)
result['algorithm'].append('apriori')
result['supp'].append(supp)
result['conf'].append(conf)
result['time'].append(end-start)

Enter min support    : 0.15
Enter min confidence : 0.3


In [248]:
freq_ap

,support,itemsets
0,0.260419,(russianukrainianwar)
1,0.209588,(stoprussia)
2,0.401590,(ukraine)
3,0.204529,(ukrainerussianwar)
4,0.240183,(ukrainewar)
5,0.232715,(украина)
6,0.202120,"(ukraine, ukrainerussianwar)"
7,0.231029,"(ukraine, ukrainewar)"
8,0.195856,"(ukrainewar, ukrainerussianwar)"
9,0.194893,"(ukraine, ukrainewar, ukrainerussianwar)"


In [249]:
rules_apriori

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ukraine),(ukrainerussianwar),0.401590,0.204529,0.202120,0.503299,2.460772,0.119983,1.601510
1,(ukrainerussianwar),(ukraine),0.204529,0.401590,0.202120,0.988221,2.460772,0.119983,50.805011
2,(ukraine),(ukrainewar),0.401590,0.240183,0.231029,0.575285,2.395193,0.134574,1.789004
3,(ukrainewar),(ukraine),0.240183,0.401590,0.231029,0.961886,2.395193,0.134574,15.700389
4,(ukrainewar),(ukrainerussianwar),0.240183,0.204529,0.195856,0.815446,3.986947,0.146732,4.310242
5,(ukrainerussianwar),(ukrainewar),0.204529,0.240183,0.195856,0.957597,3.986947,0.146732,17.919015
6,"(ukraine, ukrainewar)",(ukrainerussianwar),0.231029,0.204529,0.194893,0.843587,4.124535,0.147641,5.085711
7,"(ukraine, ukrainerussianwar)",(ukrainewar),0.202120,0.240183,0.194893,0.964243,4.014617,0.146347,21.249546
8,"(ukrainewar, ukrainerussianwar)",(ukraine),0.195856,0.401590,0.194893,0.995080,2.477851,0.116239,121.626837
9,(ukraine),"(ukrainewar, ukrainerussianwar)",0.401590,0.195856,0.194893,0.485303,2.477851,0.116239,1.562363


# 2. FP-Growth Implementation

In [250]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

supp=float(input("Enter min support    : "))
conf=float(input("Enter min confidence : "))


start = time.time()
freq_fp=fpgrowth(txn, min_support=supp,max_len=3, use_colnames=True)
end = time.time()
rules_fp=association_rules(freq_fp,metric="confidence",min_threshold=conf)
result['language'].append(language)
result['algorithm'].append('fpgrowth')
result['supp'].append(supp)
result['conf'].append(conf)
result['time'].append(end-start)


Enter min support    : 0.15
Enter min confidence : 0.3


In [251]:
freq_fp

,support,itemsets
0,0.401590,(ukraine)
1,0.232715,(украина)
2,0.240183,(ukrainewar)
3,0.204529,(ukrainerussianwar)
4,0.209588,(stoprussia)
5,0.260419,(russianukrainianwar)
6,0.231029,"(ukraine, ukrainewar)"
7,0.202120,"(ukraine, ukrainerussianwar)"
8,0.195856,"(ukrainewar, ukrainerussianwar)"
9,0.194893,"(ukraine, ukrainewar, ukrainerussianwar)"


In [252]:
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ukraine),(ukrainewar),0.401590,0.240183,0.231029,0.575285,2.395193,0.134574,1.789004
1,(ukrainewar),(ukraine),0.240183,0.401590,0.231029,0.961886,2.395193,0.134574,15.700389
2,(ukraine),(ukrainerussianwar),0.401590,0.204529,0.202120,0.503299,2.460772,0.119983,1.601510
3,(ukrainerussianwar),(ukraine),0.204529,0.401590,0.202120,0.988221,2.460772,0.119983,50.805011
4,(ukrainewar),(ukrainerussianwar),0.240183,0.204529,0.195856,0.815446,3.986947,0.146732,4.310242
5,(ukrainerussianwar),(ukrainewar),0.204529,0.240183,0.195856,0.957597,3.986947,0.146732,17.919015
6,"(ukraine, ukrainewar)",(ukrainerussianwar),0.231029,0.204529,0.194893,0.843587,4.124535,0.147641,5.085711
7,"(ukraine, ukrainerussianwar)",(ukrainewar),0.202120,0.240183,0.194893,0.964243,4.014617,0.146347,21.249546
8,"(ukrainewar, ukrainerussianwar)",(ukraine),0.195856,0.401590,0.194893,0.995080,2.477851,0.116239,121.626837
9,(ukraine),"(ukrainewar, ukrainerussianwar)",0.401590,0.195856,0.194893,0.485303,2.477851,0.116239,1.562363


# Experiments

In [253]:
experiment=[freq_ap,freq_fp]
algo=int(input("Enter 0: for selecting apriori frequent itemset\nEnter 1: for selecting fpgrowth frequent itemset\n"))

Enter 0: for selecting apriori frequent itemset
Enter 1: for selecting fpgrowth frequent itemset
0


# a) Singleton

In [254]:
experiment[algo][experiment[algo].itemsets.apply(lambda x: len(x)==1)]

,support,itemsets
0,0.260419,(russianukrainianwar)
1,0.209588,(stoprussia)
2,0.401590,(ukraine)
3,0.204529,(ukrainerussianwar)
4,0.240183,(ukrainewar)
5,0.232715,(украина)


# b) Pair

In [255]:
experiment[algo][experiment[algo].itemsets.apply(lambda x: len(x)==2)]

,support,itemsets
6,0.202120,"(ukraine, ukrainerussianwar)"
7,0.231029,"(ukraine, ukrainewar)"
8,0.195856,"(ukrainewar, ukrainerussianwar)"


# c) Triplet

In [256]:
experiment[algo][experiment[algo].itemsets.apply(lambda x: len(x)==3)]

,support,itemsets
9,0.194893,"(ukraine, ukrainewar, ukrainerussianwar)"


# Final Results

In [257]:
result_df=pd.DataFrame(result)
result_df

,language,algorithm,supp,conf,time
0,en,apriori,0.10,0.3,0.086978
1,en,fpgrowth,0.10,0.3,0.132962
2,en,apriori,0.20,0.3,0.082575
3,en,fpgrowth,0.20,0.3,0.132372
4,en,apriori,0.35,0.3,0.080137
5,en,fpgrowth,0.35,0.3,0.130474
6,en,apriori,0.40,0.3,0.084057
7,en,fpgrowth,0.40,0.3,0.127782
8,en,apriori,0.28,0.3,0.131536
9,en,fpgrowth,0.28,0.3,0.129464


In [258]:
import plotly.express as px


fig = px.scatter(result_df, x="supp",y="time",animation_frame='language',color='algorithm' ,title="time v/s support")


fig.show()